In [1]:
# Set path to data
from google.colab import drive
drive.mount('/content/gdrive')
%cd './gdrive/My Drive/Matching with RE288/Data_and_Fits'

Mounted at /content/gdrive
/content/gdrive/My Drive/Matching with RE288/Data_and_Fits


In [2]:
# Packages we'll use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from google.colab import files
import zipfile, io, os
# Load in RE288 game state function; uncomment next line if first time using
#files.upload()
#from RE288 import RE288_calc
print("Packages loaded")

Packages loaded


In [7]:
data

,Unnamed: 0,pitch_type,game_date,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,fielder_2,effective_speed,game_pk,pitcher.1,fielder_2.1,at_bat_number,pitch_number,pitch_name,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,game_state,inning_same,total_runs,change_score,runs_inn_end,RE288,RE288_change
0,0,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,field_out,hit_into_play,5.0,Brock Holt lines out to left fielder Rajai Dav...,R,L,R,CLE,BOS,X,7.0,line_drive,3.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,93.704,446867.0,545333.0,543228.0,74.0,5.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,65.0,False,8,0,0.0,1.969061,-0.016539
1,1,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,2.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,94.288,446867.0,545333.0,543228.0,74.0,4.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,61.0,True,8,0,0.0,1.952521,0.015919
2,2,FT,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,12.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,1.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,94.067,446867.0,545333.0,543228.0,74.0,3.0,2-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,57.0,True,8,0,0.0,1.968441,0.002399
3,3,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,0.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,95.092,446867.0,545333.0,543228.0,74.0,2.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Strategic,53.0,True,8,0,0.0,1.970840,-0.030190
4,4,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,foul,9.0,NaN,R,L,R,CLE,BOS,S,NaN,NaN,0.0,0.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,95.887,446867.0,545333.0,543228.0,74.0,1.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,49.0,True,8,0,2.0,1.940650,0.012106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891242,2891242,KC,2018-10-01,German Marquez,592626.0,608566.0,NaN,ball,14.0,NaN,R,L,R,LAD,COL,B,NaN,NaN,0.0,2.0,2018.0,0.0,0.0,0.0,0.0,1.0,Bot,547172.0,83.472,570335.0,608566.0,547172.0,4.0,3.0,Knuckle Curve,0.0,0.0,0.0,0.0,Infield shift,Standard,97.0,True,5,0,NaN,1.917817,NaN
2891243,2891243,FF,2018-10-01,Walker Buehler,453568.0,621111.0,NaN,called_strike,1.0,NaN,R,L,R,LAD,COL,S,NaN,NaN,1.0,0.0,2018.0,0.0,0.0,0.0,0.0,1.0,Top,518735.0,97.835,570335.0,621111.0,518735.0,1.0,2.0,4-Seam Fastball,0.0,0.0,0.0,0.0,Infield shift,Standard,34.0,True,5,0,NaN,2.020894,NaN
2891244,2891244,FF,2018-10-01,German Marquez,592626.0,608566.0,NaN,called_strike,7.0,NaN,R,L,R,LAD,COL,S,NaN,NaN,0.0,1.0,2018.0,0.0,0.0,0.0,0.0,1.0,Bot,547172.0,97.006,570335.0,608566.0,547172.0,4.0,2.0,4-Seam Fastball,0.0,0.0,0.0,0.0,Infield shift,Standard,85.0,True,5,0,NaN,1.981160,NaN
2891245,2891245,FF,2018-10-01,Walker Buehler,453568.0,621111.0,NaN,ball,11.0,NaN,R,L,R,LAD,COL,B,NaN,NaN,0.0,0.0,2018.0,0.0,0.0,0.0,0.0,1.0,Top,518735.0,97.745,570335.0,621111.0,518735.0,1.0,1.0,4-Seam Fastball,0.0,0.0,0.0,0.0,Infield shift,Standard,30.0,True,5,0,NaN,1.976912,NaN


# Turning raw data into single df, fixing NAs, and adding needed columns. 

In [ ]:
# Combine all files into single doc. 
#header = ['pitch_type','game_date','batter','pitcher','events','description',\
#          'zone','des','stand','p_throws','home_team','away_team','type','hit_location',\
#          'bb_type','balls','strikes','game_year','on_3b','on_2b','on_1b',\
#          'outs_when_up','inning','inning_topbot','fielder_2','hit_distance_sc',\
#          'launch_speed','launch_angle','effective_speed','at_bat_number',\
#          'pitch_number','home_score','away_score','if_fielding_alignment','of_fielding_alignment']

# Using glob to match the pattern 'csv'
#all_filenames = [file for file in glob.glob('*.csv')]

# Concatenate all files
#combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])

# Save the returned resulting df as .csv
#combined_csv.to_csv("All_MLB_data.csv")

In [ ]:
# Read in file
data_raw = pd.read_csv("All_MLB_data.csv")
data_raw = data_raw.drop(['Unnamed: 0'], axis=1)
data = data_raw.copy()
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,...,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,FF,2016-10-02,92.5,-1.8159,6.4706,Andrew Bailey,545358,457732,field_out,hit_into_play,NaN,NaN,NaN,NaN,3.0,Max Stassi lines out to center fielder Rafael ...,R,R,R,LAA,HOU,X,8.0,line_drive,0,0,2016,-0.018667,1.666300,0.538,2.953,NaN,514888.0,503556.0,2,9,Top,122.3,65.73,NaN,...,-13.553,3.44,1.55,330.0,88.8,26.1,90.967,2560.0,5.862,449287,457732,542208.0,596847.0,463610.0,592230.0,460060.0,527043.0,542364.0,502544.0,54.6370,0.104,0.127,0.0,1.0,0.0,0.0,3.0,73,1,4-Seam Fastball,8,1,1,8,1,8,1,8,Standard,Standard
1,FF,2016-10-02,93.9,-1.9688,6.4481,Andrew Bailey,607223,457732,strikeout,called_strike,NaN,NaN,NaN,NaN,7.0,A. J. Reed called out on strikes.,R,L,R,LAA,HOU,S,2.0,NaN,1,2,2016,0.057875,1.882733,-0.375,2.021,NaN,514888.0,503556.0,1,9,Top,NaN,NaN,NaN,...,-10.160,3.60,1.67,NaN,NaN,NaN,92.763,2550.0,6.112,449287,457732,542208.0,596847.0,463610.0,592230.0,460060.0,527043.0,542364.0,502544.0,54.3866,NaN,NaN,0.0,1.0,0.0,0.0,NaN,72,5,4-Seam Fastball,8,1,1,8,1,8,1,8,Standard,Standard
2,FF,2016-10-02,92.2,-1.8711,6.4289,Andrew Bailey,607223,457732,NaN,foul,NaN,NaN,NaN,NaN,2.0,NaN,R,L,R,LAA,HOU,S,NaN,NaN,1,2,2016,0.180342,1.402567,0.132,2.974,NaN,514888.0,503556.0,1,9,Top,NaN,NaN,NaN,...,-16.773,3.56,1.64,135.0,73.0,11.6,91.205,2630.0,5.908,449287,457732,542208.0,596847.0,463610.0,592230.0,460060.0,527043.0,542364.0,502544.0,54.5908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,4,4-Seam Fastball,8,1,1,8,1,8,1,8,Standard,Standard
3,FC,2016-10-02,87.7,-2.0108,6.3487,Andrew Bailey,607223,457732,NaN,swinging_strike,NaN,NaN,NaN,NaN,14.0,NaN,R,L,R,LAA,HOU,S,NaN,NaN,1,1,2016,0.441975,0.866500,1.326,2.419,NaN,514888.0,503556.0,1,9,Top,NaN,NaN,NaN,...,-24.405,3.56,1.64,NaN,NaN,NaN,86.389,2854.0,5.654,449287,457732,542208.0,596847.0,463610.0,592230.0,460060.0,527043.0,542364.0,502544.0,54.8450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,3,Cutter,8,1,1,8,1,8,1,8,Standard,Standard
4,CU,2016-10-02,78.1,-2.0183,6.4683,Andrew Bailey,607223,457732,NaN,blocked_ball,NaN,NaN,NaN,NaN,14.0,NaN,R,L,R,LAA,HOU,B,NaN,NaN,0,1,2016,0.621500,-0.460767,1.067,0.108,NaN,514888.0,503556.0,1,9,Top,NaN,NaN,NaN,...,-37.970,3.64,1.64,NaN,NaN,NaN,74.725,2795.0,4.623,449287,457732,542208.0,596847.0,463610.0,592230.0,460060.0,527043.0,542364.0,502544.0,55.8756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,2,Curveball,8,1,1,8,1,8,1,8,Standard,Standard


In [ ]:
# Replace null values with 0s. 
data['on_1b'].fillna(0, inplace=True)
data['on_2b'].fillna(0, inplace=True)
data['on_3b'].fillna(0, inplace=True)

# Replace runner MLB player IDs with a 1. 
data.loc[(data['on_1b'] > 0), 'on_1b'] = 1
data.loc[(data['on_2b'] > 0), 'on_2b'] = 1
data.loc[(data['on_3b'] > 0), 'on_3b'] = 1

In [ ]:
# Assign game state identifier for every pitch thrown. 288 game states possible \
# using all unique combinations of balls, strikes, outs, and location of baserunners.
game_state = RE288_calc(data=data, balls='balls', strikes='strikes', outs='outs_when_up', \
        runner_first='on_1b', runner_second='on_2b', runner_third='on_3b')

# Convert list to Pandas dataframe and concatenate with existing data
game_state = pd.DataFrame(game_state)
game_state.columns = ['game_state']
data_gs = pd.concat([data, game_state], axis=1, join='inner')

# Sort by game_date, inning, 
data_gs = data_gs.sort_values(['game_pk', 'inning', 'outs_when_up', 'pitch_number'], \
                              ascending=(True,False,False,False))
data_gs = data_gs.reset_index()

# Add column to indicate whether the inning changed after each pitch. 
data_gs['inning_same'] = data_gs.inning == data_gs.inning.shift()

In [ ]:
data_gs.to_csv('All_MLB_data_gs.csv')

In [ ]:
# Add column of total runs scored to this pitch in the inning. 
#data_gs['total_runs'] = data['home_score'] + data['away_score']
runs_scored = []
for row, value in enumerate(data_gs['game_pk']):
    curr_game = value
    df_length = len(data_gs)-1
    if (row<df_length):
        prev_row_game = row+1
        prev_row_game_val = data_gs.game_pk[prev_row_game]
        if (curr_game!=prev_row_game_val):
            runs = data_gs.home_score[row] + data_gs.away_score[row]
            runs_scored.append(runs)
        if (row==0):
            prev_row = row+1
            ho_sc_1 = data_gs.home_score[row]
            aw_sc_1 = data_gs.away_score[row]
            curr_tot = ho_sc_1 + aw_sc_1
            ho_sc_2 = data_gs.home_score[prev_row]
            aw_sc_2 = data_gs.away_score[prev_row]
            prev_tot = ho_sc_2 + aw_sc_2
            if (curr_tot < prev_tot):
                runs_scored.append(prev_tot)
            else:
                runs_scored.append(curr_tot)
        else:
            prev_row = row-1
            ho_sc_1 = data_gs.home_score[row]
            aw_sc_1 = data_gs.away_score[row]
            curr_tot = ho_sc_1 + aw_sc_1
            ho_sc_2 = data_gs.home_score[prev_row]
            aw_sc_2 = data_gs.away_score[prev_row]
            prev_tot = ho_sc_2 + aw_sc_2
            if (curr_tot > prev_tot):
                runs_scored.append(prev_tot)
            else:
                runs_scored.append(curr_tot)
    if (row==df_length):
        prev_row = row-1
        ho_sc_1 = data_gs.home_score[row]
        aw_sc_1 = data_gs.away_score[row]
        curr_tot = ho_sc_1 + aw_sc_1
        ho_sc_2 = data_gs.home_score[prev_row]
        aw_sc_2 = data_gs.away_score[prev_row]
        prev_tot = ho_sc_2 + aw_sc_2
        if (curr_tot > prev_tot):
            runs_scored.append(prev_tot)
        else:
            runs_scored.append(curr_tot)

# Append total runs col to df.
runs_scored = pd.DataFrame(runs_scored)
change_score = pd.DataFrame(runs_scored)
change_score.columns = ['total_runs']
data_gs = pd.concat([data_gs, runs_scored], axis=1)  

In [ ]:
# Add column to indicate the change in score after each pitch. 
#data_gs['inning_same'][:10]
change_score = []
for row, value in enumerate(data_gs['inning_same']):
    if (value==True):
        minus = row-1
        change_val = data_gs.total_runs[minus] - data_gs.total_runs[row]
        if change_val >= 0:
              change_score.append(change_val)
        else:
          change_score.append(0)
    else:
        change_score.append(0)

# Append the change_score col to df
change_score = pd.DataFrame(change_score)
change_score.columns = ['change_score']
data_gs = pd.concat([data_gs, change_score], axis=1)

In [ ]:
data_gs.to_csv("All_MLB_data_gs.csv")

In [ ]:
# Add column that sums number of runs scored in that inning after each pitch. 
last_false = 0
run_sum = []
run_sum_val = 0
for row, value in enumerate(data_gs['inning_same']):
    if (value==False):
        end = row-1
        run_sum_val = data_gs['change_score'].loc[last_false:end].cumsum()
        run_sum.append(run_sum_val)
        last_false = row + 1
    else:
        pass
        
all_run_sums = []
for sublist in run_sum:
    for i in sublist:
        all_run_sums.append(i)

# Append the cumulative sum of runs to the df
runs_to_inning_end = pd.DataFrame(all_run_sums)
runs_to_inning_end.columns = ['runs_inn_end']
data_gs = pd.concat([data_gs, runs_to_inning_end], axis=1)

In [ ]:
# Save new dataframe to new .csv file titled *_gs.csv
data_gs.to_csv('All_MLB_data_gs.csv')

In [ ]:
# If picking up from here. 
data_raw = pd.read_csv('All_MLB_data_gs.csv')
data_gs = data_raw.copy()
data_gs = data_gs.drop(['Unnamed: 0'], axis=1)
data_gs.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,7,10,11,17,18,19,20,21,22,23,25,38,45,80,89,90,92) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,...,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,game_state,inning_same,total_runs,change_score,runs_inn_end
0,1079167.0,FF,2016-04-05,93.9,-1.3212,5.4120,Trevor Bauer,571788.0,545333.0,field_out,hit_into_play,NaN,NaN,NaN,NaN,5.0,Brock Holt lines out to left fielder Rajai Dav...,R,L,R,CLE,BOS,X,7.0,line_drive,3.0,1.0,2016.0,-0.909333,1.574567,0.086,2.621,0.0,0.0,1.0,2.0,9.0,Top,84.27,101.22,...,18.3,93.704,2151.0,6.103,446867.0,545333.0,543228.0,435063.0,543401.0,346874.0,596019.0,434658.0,571980.0,407781.0,54.3962,0.384,0.439,0.0,1.0,0.0,0.0,4.0,74.0,5.0,4-Seam Fastball,2.0,6.0,6.0,2.0,6.0,2.0,6.0,2.0,Standard,Standard,65.0,False,8,0,0.0
1,1079168.0,FF,2016-04-05,94.9,-1.1898,5.4037,Trevor Bauer,571788.0,545333.0,NaN,ball,NaN,NaN,NaN,NaN,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,2.0,1.0,2016.0,-0.562808,1.831133,-0.522,3.543,0.0,0.0,1.0,2.0,9.0,Top,NaN,NaN,...,NaN,94.288,2339.0,6.181,446867.0,545333.0,543228.0,435063.0,543401.0,346874.0,596019.0,434658.0,571980.0,407781.0,54.3182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,4.0,4-Seam Fastball,2.0,6.0,6.0,2.0,6.0,2.0,6.0,2.0,Standard,Standard,61.0,True,8,0,0.0
2,1079169.0,FT,2016-04-05,93.8,-1.2106,5.4904,Trevor Bauer,571788.0,545333.0,NaN,ball,NaN,NaN,NaN,NaN,12.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,1.0,1.0,2016.0,-1.015100,1.343800,0.034,4.011,0.0,0.0,1.0,2.0,9.0,Top,NaN,NaN,...,NaN,94.067,2248.0,6.115,446867.0,545333.0,543228.0,435063.0,543401.0,346874.0,596019.0,434658.0,571980.0,407781.0,54.3846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,3.0,2-Seam Fastball,2.0,6.0,6.0,2.0,6.0,2.0,6.0,2.0,Standard,Standard,57.0,True,8,0,0.0
3,1079170.0,FF,2016-04-05,94.7,-1.1385,5.6268,Trevor Bauer,571788.0,545333.0,NaN,ball,NaN,NaN,NaN,NaN,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,0.0,1.0,2016.0,-0.679708,1.527267,-0.386,3.584,0.0,0.0,1.0,2.0,9.0,Top,NaN,NaN,...,NaN,95.092,2446.0,6.132,446867.0,545333.0,543228.0,435063.0,543401.0,346874.0,596019.0,434658.0,571980.0,407781.0,54.3680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,2.0,4-Seam Fastball,2.0,6.0,6.0,2.0,6.0,2.0,6.0,2.0,Standard,Strategic,53.0,True,8,0,0.0
4,1079171.0,FF,2016-04-05,95.5,-1.2638,5.4583,Trevor Bauer,571788.0,545333.0,NaN,foul,NaN,NaN,NaN,NaN,9.0,NaN,R,L,R,CLE,BOS,S,NaN,NaN,0.0,0.0,2016.0,-1.055458,1.608967,0.279,1.748,0.0,0.0,1.0,2.0,9.0,Top,NaN,NaN,...,21.0,95.887,2430.0,6.453,446867.0,545333.0,543228.0,435063.0,543401.0,346874.0,596019.0,434658.0,571980.0,407781.0,54.0470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,1.0,4-Seam Fastball,2.0,6.0,6.0,2.0,6.0,2.0,6.0,2.0,Standard,Standard,49.0,True,8,0,2.0


# Create NL and AL dataframes, and calculate RE288 for each season. 

In [ ]:
df_gs_spec = data_gs[['pitch_type','game_date','player_name','batter',\
                             'pitcher','events','description','zone','des',\
                             'game_type','stand','p_throws','home_team',\
                             'away_team','type','hit_location','bb_type','balls',\
                             'strikes','game_year','on_3b','on_2b','on_1b',\
                             'outs_when_up', 'inning','inning_topbot','fielder_2',\
                             'effective_speed','game_pk','pitcher.1','fielder_2.1',\
                             'at_bat_number','pitch_number','pitch_name',\
                             'post_away_score','post_home_score','post_bat_score',\
                             'post_fld_score','if_fielding_alignment',\
                             'of_fielding_alignment','game_state','inning_same',\
                             'total_runs','change_score','runs_inn_end']]

In [ ]:
df_gs_spec.head()

,pitch_type,game_date,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,fielder_2,effective_speed,game_pk,pitcher.1,fielder_2.1,at_bat_number,pitch_number,pitch_name,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,game_state,inning_same,total_runs,change_score,runs_inn_end
0,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,field_out,hit_into_play,5.0,Brock Holt lines out to left fielder Rajai Dav...,R,L,R,CLE,BOS,X,7.0,line_drive,3.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,93.704,446867.0,545333.0,543228.0,74.0,5.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,65.0,False,8,0,0.0
1,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,2.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,94.288,446867.0,545333.0,543228.0,74.0,4.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,61.0,True,8,0,0.0
2,FT,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,12.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,1.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,94.067,446867.0,545333.0,543228.0,74.0,3.0,2-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,57.0,True,8,0,0.0
3,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,0.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,95.092,446867.0,545333.0,543228.0,74.0,2.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Strategic,53.0,True,8,0,0.0
4,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,foul,9.0,NaN,R,L,R,CLE,BOS,S,NaN,NaN,0.0,0.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,95.887,446867.0,545333.0,543228.0,74.0,1.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,49.0,True,8,0,2.0


In [ ]:
# Drop rows where the home team is nan
df_gs_spec = df_gs_spec.dropna(subset=['home_team', 'away_team'])

In [ ]:
# Create the NL and AL dataframes
AL_teams = ['CLE', 'OAK', 'NYY', 'BAL', 'KC', 'TEX', 'TB', 'LAA', 'TOR', 'CWS', \
            'DET', 'SEA', 'BOS', 'MIN', 'HOU']
NL_teams = ['COL', 'LAD', 'NYM', 'PHI', 'SF', 'STL', 'WSH', 'CHC', 'MIA', 'PIT', \
            'SD', 'ATL', 'CIN', 'MIL', 'ARI']
AL_data = df_gs_spec.loc[df_gs_spec['home_team'].isin(AL_teams)]
NL_data = df_gs_spec.loc[df_gs_spec['home_team'].isin(NL_teams)]

In [ ]:
# Double check that 
print("AL Teams:", AL_data.home_team.unique())
print("NL Teams:", NL_data.home_team.unique())

AL Teams: ['CLE' 'OAK' 'NYY' 'BAL' 'KC' 'TEX' 'TB' 'LAA' 'TOR' 'CWS' 'DET' 'SEA'
 'BOS' 'MIN' 'HOU']
NL Teams: ['ARI' 'SD' 'CIN' 'MIL' 'PIT' 'ATL' 'MIA' 'SF' 'WSH' 'NYM' 'COL' 'CHC'
 'STL' 'PHI' 'LAD']


In [ ]:
# Isolate each game state for each year and calculate the average for NL teams
game_states = list(range(1,289))
years = list(range(2016, 2020))

# Empty lists
year_match = []
game_state_vals = []
run_expectancy = []

for i in years:
  df_year = NL_data[NL_data['game_year'] == i]
  for j in game_states:
    df_game_state = NL_data[NL_data['game_state'] == j]
    avg_rs_gs = df_game_state['runs_inn_end'].mean()
    year_match.append(i)
    game_state_vals.append(j)
    run_expectancy.append(avg_rs_gs)

# Combine the three lists into a single dataframe
RE288_NL = pd.DataFrame({'Year':year_match, 'Game_State':game_state_vals, \
                         'NL_RE288':run_expectancy})

In [ ]:
RE288_NL.describe()

,Year,Game_State,NL_RE288
count,1152.00000,1152.000000,1152.000000
mean,2017.50000,144.500000,1.971522
std,1.11852,83.174045,0.125870
min,2016.00000,1.000000,1.588496
25%,2016.75000,72.750000,1.912018
50%,2017.50000,144.500000,1.971588
75%,2018.25000,216.250000,2.026546
max,2019.00000,288.000000,2.480000


In [ ]:
# Isolate each game state for each year and calculate the average for AL teams
game_states = list(range(1,289))
years = list(range(2016, 2020))

# Empty lists
year_match = []
game_state_vals = []
run_expectancy = []

for i in years:
  df_year = AL_data[AL_data['game_year'] == i]
  for j in game_states:
    df_game_state = AL_data[AL_data['game_state'] == j]
    avg_rs_gs = df_game_state['runs_inn_end'].mean()
    year_match.append(i)
    game_state_vals.append(j)
    run_expectancy.append(avg_rs_gs)

# Combine the three lists into a single dataframe
RE288_AL = pd.DataFrame({'Year':year_match, 'Game_State':game_state_vals, \
                         'AL_RE288':run_expectancy})

In [ ]:
RE288_AL.describe()

,Year,Game_State,AL_RE288
count,1152.00000,1152.000000,1152.000000
mean,2017.50000,144.500000,1.972405
std,1.11852,83.174045,0.150293
min,2016.00000,1.000000,1.452381
25%,2016.75000,72.750000,1.906387
50%,2017.50000,144.500000,1.957486
75%,2018.25000,216.250000,2.025187
max,2019.00000,288.000000,2.616000


In [ ]:
# Save these datraframes
RE288_NL.to_csv('RE288_NL.csv')
RE288_AL.to_csv('RE288_AL.csv')

# Assign RE288 to all observations and save resulting df using 10 distributed notebooks to speed things up. 

In [ ]:
# Load dfs if starting from here
data_raw = pd.read_csv('All_MLB_data_gs.csv')
df_gs_spec = data_raw.copy()
df_gs_spec = df_gs_spec.drop(['Unnamed: 0', 'RE288'], axis=1)
RE288_AL = pd.read_csv('RE288_AL.csv')
RE288_AL = RE288_AL.drop(['Unnamed: 0'], axis=1)
RE288_NL = pd.read_csv('RE288_NL.csv')
RE288_NL = RE288_NL.drop(['Unnamed: 0'], axis=1)

In [ ]:
# Create AL and NL team lists
AL_teams = ['CLE', 'OAK', 'NYY', 'BAL', 'KC', 'TEX', 'TB', 'LAA', 'TOR', 'CWS', \
            'DET', 'SEA', 'BOS', 'MIN', 'HOU']
NL_teams = ['COL', 'LAD', 'NYM', 'PHI', 'SF', 'STL', 'WSH', 'CHC', 'MIA', 'PIT', \
            'SD', 'ATL', 'CIN', 'MIL', 'ARI']

In [ ]:
# Assign the average run expectancy for each pitch in the df. 
RE288_col = [] # empty list to store our data
RE_range = list(range(len(RE288_NL)))
df_len = list(range(0, 289125))

for i in df_len:
  # Get the game state and year
  gs = df_gs_spec['game_state'][i]
  yr = df_gs_spec['game_year'][i]
  # Find the corresponding RE288
  if (df_gs_spec['home_team'][i] in NL_teams):
    for j in RE_range:
      if RE288_NL['Year'][j]==yr:
        if RE288_NL['Game_State'][j]==gs:
          val = RE288_NL['NL_RE288'][j]
          RE288_col.append(val) # append to col
  else:
    for j in RE_range:
      if RE288_AL['Year'][j]==yr:
        if RE288_AL['Game_State'][j]==gs:
          val = RE288_AL['AL_RE288'][j]
          RE288_col.append(val) # append to col
  if i%1000 == 0:
    print(i, "observations completed")
  if i%5000 == 0:
    save = pd.DataFrame(RE288_col)
    save.to_csv('RE288_0_289125.csv')

print(len(df_gs_spec))
print(len(RE288_col))

0 observations completed
1000 observations completed
2000 observations completed
3000 observations completed
4000 observations completed
5000 observations completed
6000 observations completed
7000 observations completed
8000 observations completed
9000 observations completed
10000 observations completed
11000 observations completed
12000 observations completed
13000 observations completed
14000 observations completed
15000 observations completed
16000 observations completed
17000 observations completed
18000 observations completed
19000 observations completed
20000 observations completed
21000 observations completed
22000 observations completed
23000 observations completed
24000 observations completed
25000 observations completed
26000 observations completed
27000 observations completed
28000 observations completed
29000 observations completed
30000 observations completed
31000 observations completed
32000 observations completed
33000 observations completed
34000 observations complete

In [ ]:
val10 = 289125
print(val10, 2*val10, 3*val10, 4*val10, 5*val10, 6*val10, 7*val10, 8*val10, 9*val10)


289125 578250 867375 1156500 1445625 1734750 2023875 2313000 2602125


In [ ]:
# Combine all the batched files together into a single column
# Read in the data
first = pd.read_csv('RE288_1st_batch.csv')
sec = pd.read_csv('RE288_2nd_batch.csv')
third = pd.read_csv('RE288_3rd_batch.csv')
four = pd.read_csv('RE288_4th_batch.csv')
five = pd.read_csv('RE288_5th_batch.csv')
six = pd.read_csv('RE288_6th_batch.csv')
sev = pd.read_csv('RE288_7th_batch.csv')
eight = pd.read_csv('RE288_8th_batch.csv')
nine = pd.read_csv('RE288_9th_batch.csv')
ten = pd.read_csv('RE288_10th_batch.csv')

# Combine the cols
all_RE288 = pd.concat([first, sec, third, four, five, six, sev, eight, \
                       nine, ten], ignore_index=True)
all_RE288 = all_RE288.drop(['Unnamed: 0'], axis=1)
all_RE288.columns = ['RE288']
all_RE288

,RE288
0,1.969061
1,1.952521
2,1.968441
3,1.970840
4,1.940650
...,...
2891242,1.917817
2891243,2.020894
2891244,1.981160
2891245,1.976912


In [ ]:
# Add new RE288 col to the dataframe
df_gs_spec['RE288'] = all_RE288['RE288']

In [ ]:
# Save the df
df_gs_spec.to_csv('All_MLB_data_RE288.csv')

# Calculate the change in RE288 following each pitch

In [ ]:
# Add column to indicate the change in RE288 after each pitch. 
#data_gs['inning_same'][:10]
change_RE288 = []
for row, value in enumerate(df_gs_spec['inning_same']):
    if (value==True):
        minus = row-1
        change_val = df_gs_spec.RE288[minus] - df_gs_spec.RE288[row]
        change_RE288.append(change_val)

# Append the change_score col to df
change_RE288 = pd.DataFrame(change_RE288)
change_RE288.columns = ['change_RE28']
change_RE288.describe()
df_gs_spec = pd.concat([df_gs_spec, change_RE288], axis=1)

,change_RE28
count,2.802126e+06
mean,-1.516086e-05
std,7.501908e-02
min,-9.228346e-01
25%,-2.946896e-02
50%,0.000000e+00
75%,3.018950e-02
max,9.866294e-01


In [ ]:
# Reverse the sign for change in RE288 (i.e., a negative is good) 
# and add to df_gs_spec
sign_flip = change_RE288*-1
df_gs_spec['RE288_change'] = sign_flip
df_gs_spec.head(10)

,pitch_type,game_date,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,fielder_2,effective_speed,game_pk,pitcher.1,fielder_2.1,at_bat_number,pitch_number,pitch_name,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,game_state,inning_same,total_runs,change_score,runs_inn_end,RE288,RE288_change
0,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,field_out,hit_into_play,5.0,Brock Holt lines out to left fielder Rajai Dav...,R,L,R,CLE,BOS,X,7.0,line_drive,3.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,93.704,446867.0,545333.0,543228.0,74.0,5.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,65.0,False,8,0,0.0,1.969061,-0.016539
1,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,2.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,94.288,446867.0,545333.0,543228.0,74.0,4.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,61.0,True,8,0,0.0,1.952521,0.015919
2,FT,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,12.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,1.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,94.067,446867.0,545333.0,543228.0,74.0,3.0,2-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,57.0,True,8,0,0.0,1.968441,0.002399
3,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,ball,11.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,0.0,1.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,95.092,446867.0,545333.0,543228.0,74.0,2.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Strategic,53.0,True,8,0,0.0,1.970840,-0.030190
4,FF,2016-04-05,Trevor Bauer,571788.0,545333.0,NaN,foul,9.0,NaN,R,L,R,CLE,BOS,S,NaN,NaN,0.0,0.0,2016.0,0.0,0.0,1.0,2.0,9.0,Top,543228.0,95.887,446867.0,545333.0,543228.0,74.0,1.0,4-Seam Fastball,6.0,2.0,6.0,2.0,Standard,Standard,49.0,True,8,0,2.0,1.940650,0.012106
5,FC,2016-04-05,Trevor Bauer,120074.0,545333.0,home_run,hit_into_play_score,14.0,David Ortiz homers (1) on a line drive to righ...,R,L,R,CLE,BOS,X,NaN,line_drive,2.0,2.0,2016.0,0.0,0.0,1.0,1.0,9.0,Top,543228.0,91.325,446867.0,545333.0,543228.0,71.0,5.0,Cutter,4.0,2.0,4.0,2.0,Infield shift,Standard,50.0,True,6,2,2.0,1.952757,-0.012106
6,FT,2016-04-05,Trevor Bauer,120074.0,545333.0,NaN,swinging_strike,14.0,NaN,R,L,R,CLE,BOS,S,NaN,NaN,2.0,1.0,2016.0,0.0,0.0,1.0,1.0,9.0,Top,543228.0,94.898,446867.0,545333.0,543228.0,71.0,4.0,2-Seam Fastball,4.0,2.0,4.0,2.0,Infield shift,Standard,49.0,True,6,0,2.0,1.940650,-0.006607
7,CU,2016-04-05,Trevor Bauer,543768.0,545333.0,strikeout,swinging_strike_blocked,14.0,Travis Shaw strikes out swinging.,R,L,R,CLE,BOS,S,2.0,NaN,0.0,2.0,2016.0,0.0,0.0,1.0,1.0,9.0,Top,543228.0,78.433,446867.0,545333.0,543228.0,73.0,3.0,Curveball,6.0,2.0,6.0,2.0,Standard,Standard,13.0,True,6,0,2.0,1.934044,0.100194
8,CH,2016-04-05,Trevor Bauer,120074.0,545333.0,NaN,ball,13.0,NaN,R,L,R,CLE,BOS,B,NaN,NaN,1.0,1.0,2016.0,0.0,0.0,1.0,1.0,9.0,Top,543228.0,87.757,446867.0,545333.0,543228.0,71.0,3.0,Changeup,4.0,2.0,4.0,2.0,Infield shift,Standard,46.0,True,6,0,2.0,2.034237,-0.117571
9,FC,2016-04-05,Trevor Bauer,543768.0,545333.0,NaN,foul,5.0,NaN,R,L,R,CLE,BOS,S,NaN,NaN,0.0,1.0,2016.0,0.0,0.0,1.0,1.0,9.0,Top,543228.0,89.788,446867.0,545333.0,543228.0,73.0,2.0,Cutter,6.0,2.0,6.0,2.0,Standard,Standard,59.0,True,6,0,2.0,1.916667,-0.007078


In [ ]:
# Save resulting dataframe
df_gs_spec.to_csv('All_MLB_data_for_GME.csv')